<a href="https://colab.research.google.com/github/rennydoig/ssc22-case-comp/blob/main/Daisy_dir/Final_LMM_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import csv
!pip install geopandas rtree
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import unary_union
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import pickle
import datetime
!pip install pysal contextily &> /dev/null
!pip install pandana
import matplotlib.pyplot as plt
import seaborn
from pysal.viz import splot
from splot.esda import plot_moran
import contextily
from pysal.explore import esda
from pysal.lib import weights
drive.mount('/content/drive')
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 12.8 MB/s 
     |████████████████████████████████| 1.0 MB 59.9 MB/s 
     |████████████████████████████████| 16.7 MB 388 kB/s 
     |████████████████████████████████| 6.3 MB 41.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 15.0 MB/s 
     |████████████████████████████████| 4.3 MB 51.9 MB/s 
  Attempting uninstall: tables
    Found existing installation: tables 3.7.0
    Uninstalling tables-3.7.0:
      Successfully uninstalled tables-3.7.0


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
/usr/local/lib/python3.7/dist-packages/spaghetti/network.py:36: FutureWarning: The next major release of pysal/spaghetti (2.0.0) will drop support for all ``libpysal.cg`` geometries. This change is a first step in refactoring ``spaghetti`` that is expected to result in dramatically reduced runtimes for network instantiation and operations. Users currently requiring network and point pattern input as ``libpysal.cg`` geometries should prepare for this simply by converting to ``shapely`` geometries.
  warnings.warn(f"{dep_msg}", FutureWarning)


Mounted at /content/drive


In [1]:
# Residual plot
# performance = pd.DataFrame()
# performance["residuals"] = result.resid.values
# performance["actual"] = data_train['avg_u_mbps']
# performance["fitted"] = result.fittedvalues
# ax = sns.residplot(x = "fitted", y = "residuals", data = performance, lowess=True)
# ax.set(ylabel='residuals')
# plt.show()

def mixedEffctModel(data, formula, random_effect):
  model = smf.mixedlm(formula, data, groups=data[random_effect])

  result = model.fit()
  print(result.summary())
  return result

def predResult(data_test, true, model_fit, random_effect, threshold):
  fix_eff_pred = result.predict(exog=data_test)
  
  # The prediction on random effects
  ran_eff_pred = []
  cd = data_test['CDUID']
  for x in cd:
    ran_eff_pred.append(result.random_effects[x])

  # Add the fixed and random effects to get the overall prediction
  pred = fix_eff_pred + np.concatenate(ran_eff_pred)

  # Get RMSE
  print("RMSE: {}".format(np.sqrt(metrics.mean_squared_error(true, pred))))

  # Get accuracy
  pred_class = [int(p >= threshold) for p in pred]
  true_class = [int(p >= threshold) for p in true]
  print("Accuracy: {}".format(accuracy_score(true_class, pred_class, normalize=False)/float(len(true_class))))

  # Get confusion matrix
  print(metrics.confusion_matrix(true_class, pred_class))

  # Get classification report
  print(classification_report(true_class,pred_class))

  # Return prediction results
  data_test['Pred'] = pred
  data_test['Pred_class'] = pred_class
  data_test['True_class'] = true_class
  data_test['Pred_accuracy'] = abs(pred-true)

def addTime(data,test=False):
  t_vals = sorted(data['date'].unique()) # sorts year-quarter values
  t_ints = [*range(1,len(t_vals)+1)] # creates a list of integers [1, 2, ...]
  if test:
    data['Time'] = np.ones(data.shape[0]) * 12
  else:
    data['Time'] = data['date'].replace(to_replace=t_vals, value=t_ints)
  

def addGroupID(data):
  data['x'] = data['geometry'].centroid.x
  data['y'] = data['geometry'].centroid.y
  coords = data[['x','y']].to_numpy()
  dbscan = DBSCAN(eps=100000, min_samples=1) # groups ‘densely grouped’ data points into a single cluster
  clusters = dbscan.fit(coords)
  labels = pd.Series(clusters.labels_) + 1
  labels.index = range(1, len(labels) + 1)
  data['GroupID'] = list(labels)
  #plt.scatter(train['x'], train['y'], c = labels)

In [3]:
os.chdir('/content/drive/MyDrive/2022 SSC Case Study/')
train = pd.read_csv('canada_test_imp.csv')
test = pd.read_csv('canada_valid_imp.csv')
os.chdir('/content/drive/MyDrive/2022 SSC Case Study/dataset/')
boundary_dat = gpd.read_file('./boundaries/statcan-census-division/statcan-census-division.shp')
display(train.head(3))

,date,PRUID,CDUID,SACTYPE,is_rural,conn_type,avg_d_mbps_wt,avg_u_mbps_wt,avg_lat_ms,tests,DA_POP,imputed
0,2019-01-01,10,1001,1,False,fixed,110.345156,72.589665,10.505591,6618,550285.0,False
1,2019-01-01,10,1001,1,False,mobile,92.795777,28.836200,65.107692,130,43085.0,False
2,2019-01-01,10,1001,1,True,fixed,133.453020,93.332288,7.532551,937,180645.0,False


In [5]:
# # Merge with boundary data to add polygon 
# boundary_dat['CDUID'] = boundary_dat['CDUID'].astype(int)
# col = ['date','CDUID','SACTYPE','is_rural','conn_type','avg_d_mbps_wt','avg_u_mbps_wt','avg_lat_ms','tests','DA_POP','imputed']
# # Training set
# train = pd.merge(train[col], boundary_dat, on='CDUID', how='left')
# train = gpd.GeoDataFrame(train, geometry='geometry', crs="EPSG:4326")
# # Testing set
# test = pd.merge(test[col], boundary_dat, on='CDUID', how='left')
# test = gpd.GeoDataFrame(test, geometry='geometry', crs="EPSG:4326")

In [5]:
#addGroupID(test)
addTime(test,test=True)
#addGroupID(train)
addTime(train)

### Predicting download speed for 2021Q4

In [6]:
result = mixedEffctModel(train, 'avg_d_mbps_wt ~ Time + SACTYPE + conn_type + is_rural', 'CDUID')
predResult(test, test['avg_d_mbps_wt'], result, 'CDUID', 50)

              Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   avg_d_mbps_wt
No. Observations:    30481     Method:               REML         
No. Groups:          293       Scale:                1287.0431    
Min. group size:     11        Likelihood:           -152827.8725 
Max. group size:     220       Converged:            Yes          
Mean group size:     104.0                                        
------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
------------------------------------------------------------------
Intercept            94.572    1.305  72.477 0.000  92.015  97.130
conn_type[T.mobile] -12.702    0.412 -30.864 0.000 -13.508 -11.895
is_rural[T.True]    -31.300    0.443 -70.704 0.000 -32.168 -30.432
Time                  3.401    0.065  52.336 0.000   3.273   3.528
SACTYPE              -6.066    0.152 -39.900 0.000  -6.364  -5.768
Group Var 

### Predicting upload speed for 2021Q4

In [8]:
result = mixedEffctModel(train, 'avg_u_mbps_wt ~ Time + SACTYPE + conn_type + is_rural', 'CDUID')
predResult(test, test['avg_u_mbps_wt'], result, 'CDUID', 10)

              Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   avg_u_mbps_wt
No. Observations:    30481     Method:               REML         
No. Groups:          293       Scale:                380.7054     
Min. group size:     11        Likelihood:           -134280.0727 
Max. group size:     220       Converged:            Yes          
Mean group size:     104.0                                        
------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
------------------------------------------------------------------
Intercept            38.315    0.951  40.298 0.000  36.452  40.179
conn_type[T.mobile] -17.611    0.224 -78.681 0.000 -18.049 -17.172
is_rural[T.True]     -9.387    0.241 -38.986 0.000  -9.859  -8.916
Time                  1.035    0.035  29.280 0.000   0.966   1.104
SACTYPE              -1.816    0.083 -21.883 0.000  -1.979  -1.654
GroupID   

### Predicting download speed for 2019-2030

In [7]:
os.chdir('/content/drive/MyDrive/2022 SSC Case Study/')
df = pd.read_csv('canada_aggregation_CD_SACTYPE_isrural_conn_type_imputed_step3.csv')
pred_d_arima = pd.read_csv('pred_download_ARIMA_2019_2030.csv')
pred_d_lmm = {'PRUID': pred_d_arima['PRUID'],
              'CDUID': pred_d_arima['CDUID'],
              'SACTYPE': pred_d_arima['SACTYPE'],
              'is_rural': pred_d_arima['is_rural'],
              'conn_type': pred_d_arima['conn_type'],}
pred_d_lmm = pd.DataFrame(pred_d_lmm)
display(pred_d_lmm.head(3))

colnames=list(pred_d_arima.columns)[5:53]
time = np.arange(48) + 1
addTime(df)
result = mixedEffctModel(df, 'avg_d_mbps_wt ~ Time + SACTYPE + conn_type + is_rural', 'CDUID')
for t in time:
  #pred_d_lmm['Time'] = np.ones(pred_d_lmm.shape[0]) * t
  pred_d_lmm['Time'] = t
  fix_eff_pred = result.predict(exog=pred_d_lmm)
  ran_eff_pred = []
  cd = pred_d_lmm['CDUID']
  for x in cd:
    ran_eff_pred.append(result.random_effects[x])
  pred = fix_eff_pred + np.concatenate(ran_eff_pred)
  col = colnames[t-1]
  pred_d_lmm[col] = pred

,PRUID,CDUID,SACTYPE,is_rural,conn_type
0,10,1001,1,False,fixed
1,10,1001,1,False,mobile
2,10,1001,1,True,fixed


              Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   avg_d_mbps_wt
No. Observations:    33252     Method:               REML         
No. Groups:          293       Scale:                1386.8485    
Min. group size:     12        Likelihood:           -167926.8964 
Max. group size:     240       Converged:            Yes          
Mean group size:     113.5                                        
------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
------------------------------------------------------------------
Intercept            95.209    1.310  72.676 0.000  92.641  97.776
conn_type[T.mobile] -15.514    0.409 -37.930 0.000 -16.315 -14.712
is_rural[T.True]    -31.522    0.440 -71.644 0.000 -32.385 -30.660
Time                  3.740    0.059  63.213 0.000   3.624   3.856
SACTYPE              -6.204    0.151 -41.023 0.000  -6.500  -5.908
Group Var 

In [6]:
pred_d_lmm=pred_d_lmm.drop(['Time'], axis=1)
display(pred_d_lmm.head(3))
pred_d_lmm.to_csv('pred_download_LMM_2019_2030.csv', index=False)

,PRUID,CDUID,SACTYPE,is_rural,conn_type,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,...,2028Q3,2028Q4,2029Q1,2029Q2,2029Q3,2029Q4,2030Q1,2030Q2,2030Q3,2030Q4
0,10,1001,1,False,fixed,118.320586,122.060282,125.799978,129.539674,133.279370,...,260.429028,264.168724,267.908419,271.648115,275.387811,279.127507,282.867203,286.606898,290.346594,294.086290
1,10,1001,1,False,mobile,102.807037,106.546733,110.286428,114.026124,117.765820,...,244.915478,248.655174,252.394870,256.134566,259.874262,263.613957,267.353653,271.093349,274.833045,278.572741
2,10,1001,1,True,fixed,86.798111,90.537807,94.277503,98.017198,101.756894,...,228.906552,232.646248,236.385944,240.125640,243.865336,247.605031,251.344727,255.084423,258.824119,262.563815


### Predicting upload speed for 2019-2030

In [16]:
os.chdir('/content/drive/MyDrive/2022 SSC Case Study/')
df = pd.read_csv('canada_aggregation_CD_SACTYPE_isrural_conn_type_imputed_step3.csv')
pred_u_arima = pd.read_csv('pred_upload_ARIMA_2019_2030.csv')
pred_u_lmm = {'PRUID': pred_u_arima['PRUID'],
              'CDUID': pred_u_arima['CDUID'],
              'SACTYPE': pred_u_arima['SACTYPE'],
              'is_rural': pred_u_arima['is_rural'],
              'conn_type': pred_u_arima['conn_type'],}
pred_u_lmm = pd.DataFrame(pred_u_lmm)
display(pred_u_lmm.head(3))

colnames=list(pred_u_arima.columns)[5:53]
time = np.arange(48) + 1
addTime(df)
result = mixedEffctModel(df, 'avg_u_mbps_wt ~ Time + SACTYPE + conn_type + is_rural', 'CDUID')
for t in time:
  pred_u_lmm['Time'] = np.ones(pred_u_lmm.shape[0]) * t
  fix_eff_pred = result.predict(exog=pred_u_lmm)
  ran_eff_pred = []
  cd = pred_u_lmm['CDUID']
  for x in cd:
    ran_eff_pred.append(result.random_effects[x])
  pred = fix_eff_pred + np.concatenate(ran_eff_pred)
  col = colnames[t-1]
  pred_u_lmm[col] = pred

,PRUID,CDUID,SACTYPE,is_rural,conn_type
0,10,1001,1,False,fixed
1,10,1001,1,False,mobile
2,10,1001,1,True,fixed


              Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   avg_u_mbps_wt
No. Observations:    33252     Method:               REML         
No. Groups:          293       Scale:                410.2339     
Min. group size:     12        Likelihood:           -147697.0872 
Max. group size:     240       Converged:            Yes          
Mean group size:     113.5                                        
------------------------------------------------------------------
                     Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
------------------------------------------------------------------
Intercept            37.177    0.743  50.059 0.000  35.722  38.633
conn_type[T.mobile] -18.812    0.222 -84.565 0.000 -19.248 -18.376
is_rural[T.True]     -9.478    0.239 -39.601 0.000  -9.947  -9.009
Time                  1.174    0.032  36.487 0.000   1.111   1.237
SACTYPE              -1.946    0.082 -23.603 0.000  -2.108  -1.785
Group Var 

In [10]:
pred_u_lmm=pred_u_lmm.drop(['Time'], axis=1)
display(pred_u_lmm.head(3))
pred_u_lmm.to_csv('pred_upload_LMM_2019_2030.csv', index=False)

,PRUID,CDUID,SACTYPE,is_rural,conn_type,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,...,2028Q3,2028Q4,2029Q1,2029Q2,2029Q3,2029Q4,2030Q1,2030Q2,2030Q3,2030Q4
0,10,1001,1,False,fixed,45.849747,47.023741,48.197735,49.371730,50.545724,...,90.461532,91.635526,92.809521,93.983515,95.157510,96.331504,97.505498,98.679493,99.853487,101.027481
1,10,1001,1,False,mobile,27.038035,28.212030,29.386024,30.560019,31.734013,...,71.649821,72.823815,73.997810,75.171804,76.345799,77.519793,78.693787,79.867782,81.041776,82.215770
2,10,1001,1,True,fixed,36.371963,37.545958,38.719952,39.893946,41.067941,...,80.983749,82.157743,83.331738,84.505732,85.679726,86.853721,88.027715,89.201710,90.375704,91.549698


In [33]:
# Ensemble two approaches
pred_d_ens = {'PRUID': pred_d_arima['PRUID'],
              'CDUID': pred_d_arima['CDUID'],
              'SACTYPE': pred_d_arima['SACTYPE'],
              'is_rural': pred_d_arima['is_rural'],
              'conn_type': pred_d_arima['conn_type'],}
pred_d_ens = pd.DataFrame(pred_d_ens)
pred_d_ens.head(3)
for col in colnames:
  a = []
  for p_arima, p_lmm in zip(pred_d_arima[col],pred_d_lmm[col]):
    if p_arima <= 50:
      a.append(p_arima)
    else:
      a.append(0.5*p_arima + 0.5*p_lmm)
  pred_d_ens[col] = a

pred_u_ens = {'PRUID': pred_u_arima['PRUID'],
              'CDUID': pred_u_arima['CDUID'],
              'SACTYPE': pred_u_arima['SACTYPE'],
              'is_rural': pred_u_arima['is_rural'],
              'conn_type': pred_u_arima['conn_type'],}
pred_u_ens = pd.DataFrame(pred_u_ens)
pred_u_ens.head(3)
for col in colnames:
  a = []
  for p_arima, p_lmm in zip(pred_u_arima[col],pred_u_lmm[col]):
    if p_arima <= 10:
      a.append(p_arima)
    else:
      a.append(0.5*p_arima + 0.5*p_lmm)
  pred_u_ens[col] = a

pred_d_ens.to_csv('pred_download_ENS_2019_2030.csv', index=False)
pred_u_ens.to_csv('pred_upload_ENS_2019_2030.csv', index=False)

In [10]:
# Download speed
pred_d_ens = pd.read_csv('pred_download_ENS_2019_2030.csv')
pred_u_ens = pd.read_csv('pred_upload_ENS_2019_2030.csv')

pred = pred_d_ens['2021Q4']
true = df[df['Time']==12]['avg_d_mbps_wt']
print("RMSE: {}".format(np.sqrt(metrics.mean_squared_error(true, pred))))
pred_class = [int(p >= 50) for p in pred]
true_class = [int(p >= 50) for p in true]
print("Accuracy: {}".format(accuracy_score(true_class, pred_class, normalize=False)/float(len(true_class))))
print(metrics.confusion_matrix(true_class, pred_class))
print(classification_report(true_class,pred_class))

RMSE: 21.59845173666471
Accuracy: 0.9895344640923854
[[ 778    5]
 [  24 1964]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       783
           1       1.00      0.99      0.99      1988

    accuracy                           0.99      2771
   macro avg       0.98      0.99      0.99      2771
weighted avg       0.99      0.99      0.99      2771



In [11]:
# Upload speed
pred = pred_u_ens['2021Q4']
true = df[df['Time']==12]['avg_u_mbps_wt']
print("RMSE: {}".format(np.sqrt(metrics.mean_squared_error(true, pred))))
pred_class = [int(p >= 10) for p in pred]
true_class = [int(p >= 10) for p in true]
print("Accuracy: {}".format(accuracy_score(true_class, pred_class, normalize=False)/float(len(true_class))))
print(metrics.confusion_matrix(true_class, pred_class))
print(classification_report(true_class,pred_class))

RMSE: 12.603351278135737
Accuracy: 0.9801515698303861
[[ 881    1]
 [  54 1835]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       882
           1       1.00      0.97      0.99      1889

    accuracy                           0.98      2771
   macro avg       0.97      0.99      0.98      2771
weighted avg       0.98      0.98      0.98      2771



In [14]:
df[df['Time']==12]

,date,PRUID,CDUID,SACTYPE,is_rural,conn_type,avg_d_mbps_wt,avg_u_mbps_wt,avg_lat_ms,tests,DA_POP,imputed,Time
30443,2021-10-01,10,1001,1,False,fixed,206.369578,102.525161,11.942926,9584,640785.0,False,12
30444,2021-10-01,10,1001,1,False,mobile,140.187612,24.954678,38.949468,376,111080.0,False,12
30445,2021-10-01,10,1001,1,True,fixed,214.096638,110.849005,11.619347,1592,273045.0,False,12
30446,2021-10-01,10,1001,1,True,mobile,146.146273,17.026045,37.272727,44,25870.0,False,12
30447,2021-10-01,10,1001,3,False,fixed,231.132254,113.201756,7.407960,201,30015.0,False,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33209,2021-10-01,62,6204,8,True,mobile,85.233333,19.181833,15.000000,6,7740.0,False,12
33210,2021-10-01,62,6205,8,False,fixed,13.033437,1.963729,9.604167,48,51385.0,True,12
33211,2021-10-01,62,6205,8,False,mobile,28.588914,10.210486,34.771429,35,29800.0,True,12
33212,2021-10-01,62,6205,8,True,mobile,85.233333,19.181833,15.000000,6,7740.0,True,12


In [15]:
pred_u_ens

,PRUID,CDUID,SACTYPE,is_rural,conn_type,2019Q1,2019Q2,2019Q3,2019Q4,2020Q1,...,2028Q3,2028Q4,2029Q1,2029Q2,2029Q3,2029Q4,2030Q1,2030Q2,2030Q3,2030Q4
0,10,1001,1,False,fixed,59.219706,64.487544,61.765747,62.640414,62.305194,...,95.801080,96.388090,96.975079,97.562082,98.149075,98.736075,99.323070,99.910069,100.497065,101.084063
1,10,1001,1,False,mobile,27.937118,25.488746,23.508234,25.302388,25.483364,...,48.383861,48.970858,49.557855,50.144853,50.731850,51.318847,51.905844,52.492841,53.079838,53.666836
2,10,1001,1,True,fixed,64.852126,61.766971,59.510523,55.786927,58.794514,...,95.932288,96.519286,97.106283,97.693280,98.280277,98.867274,99.454271,100.041269,100.628266,101.215263
3,10,1001,1,True,mobile,31.095876,18.320623,20.392904,16.863691,18.784273,...,40.171144,40.758135,41.345136,41.932131,42.519129,43.106126,43.693123,44.280120,44.867118,45.454115
4,10,1001,3,False,fixed,43.402277,55.104474,58.896995,60.433611,62.765352,...,105.402816,105.989813,106.576810,107.163807,107.750804,108.337802,108.924799,109.511796,110.098793,110.685790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766,62,6204,8,True,mobile,3.896000,9.693333,5.953000,7.396857,4.337469,...,26.032053,26.619051,27.206048,27.793045,28.380042,28.967039,29.554037,30.141034,30.728031,31.315028
2767,62,6205,8,False,fixed,2.291000,34.026596,6.900000,18.855590,1.479800,...,39.400491,39.987489,40.574486,41.161483,41.748480,42.335477,42.922474,43.509472,44.096469,44.683466
2768,62,6205,8,False,mobile,2.291000,6.900000,8.862737,7.933809,11.888700,...,29.127427,29.714425,30.301422,30.888419,31.475416,32.062413,32.649411,33.236408,33.823405,34.410402
2769,62,6205,8,True,mobile,3.896000,6.597733,3.538346,2.527618,6.700840,...,28.316629,28.903626,29.490623,30.077620,30.664618,31.251615,31.838612,32.425609,33.012606,33.599603
